### Kaustav Vats (2016048)

In [1]:
import numpy as np
import nltk
import json
import copy

In [2]:
# Load Grammer
Grammar = nltk.data.load("grammars/large_grammars/atis.cfg")
old_productions = Grammar.productions()
print(Grammar)
# Load Raw Sentences
Sentences = nltk.data.load("grammars/large_grammars/atis_sentences.txt")
# print(Sentences)

# Extract test sentences
# Sentences_Test = nltk.parse.util.extract_test_sentences(Sentences)
# print(Sentences_Test)
# parser = nltk.parse.BottomUpChartParser(Grammar)
# for sent in Sentences_Test:
#     print(sent)
# #     try:
# #         parser.chart_parse(sent[0])
# #     except:
# #         print("Unknown Word")

Grammar with 5517 productions (start state = SIGMA)
    ABBCL_NP -> QUANP_DTI QUANP_DTI QUANP_CD AJP_JJ NOUN_NP PRPRTCL_VBG
    ADJ_ABL -> only
    ADJ_ABL -> such
    ADJ_AP -> pt_adj_ap
    ADJ_AP -> other
    ADJ_AP -> last
    ADJ_AP -> less
    ADJ_AT -> the
    ADJ_AT -> a
    ADJ_AT -> no
    ADJ_AT -> pt_adj_at
    ADJ_CD -> seven
    ADJ_CD -> three hundred
    ADJ_CD -> sixty seven
    ADJ_CD -> one
    ADJ_CD -> fifty
    ADJ_CD -> eighty three
    ADJ_CD -> two hundred
    ADJ_CD -> thirty
    ADJ_CD -> eighty two
    ADJ_CD -> ten
    ADJ_CD -> fourteen
    ADJ_CD -> forty six
    ADJ_CD -> nine
    ADJ_CD -> hundred
    ADJ_CD -> five
    ADJ_CD -> twenty
    ADJ_CD -> sixty five
    ADJ_CD -> seventeen
    ADJ_CD -> eight
    ADJ_CD -> six
    ADJ_CD -> twenty five
    ADJ_CD -> forty seven
    ADJ_CD -> twelve
    ADJ_CD -> pt_adj_cd
    ADJ_CD -> three
    ADJ_CD -> two
    ADJ_CD -> four
    ADJ_CD -> eleven
    ADJ_DT -> this
    ADJ_DT -> that
    ADJ_DTI -> zero
  

In [23]:
def save(dic):
    with open('productions.json', 'w') as fp:
        json.dump(dic, fp)

def pre_process_grammar(prod):
    nprod = {}
    ustate = []
    for i in range(len(prod)):
        k = str(prod[i].lhs())
        v = list(prod[i].rhs())
#         print(v)
        
        for i in range(len(v)):
            if isinstance(v[i], str) or isinstance(v[i], bytes):
                v[i] = "'" + v[i] + "'"
            else:
                v[i] = str(v[i])
        
        if k not in ustate:
            ustate.append(k)
            nprod[k] = [v]
        else:
            if v not in nprod[k]:
                 nprod[k].append(v)
    save(nprod)
#     print(nprod)
    return nprod

def match(dic, label, x, y):
    for i in range(len(dic)):
        if dic[i][0] == x and dic[i][1] == y:
            return i
        if dic[i][0] == y and dic[i][1] == x:
            return i
    return -1

def RemoveLargeRules(old_prod):
    temp_prod = copy.deepcopy(old_prod)
    count = 0
    dic = []
    label = []
#     flag = True
#     while flag:
#         flag = false 
    for k in temp_prod:
        v = old_prod[k]
#         print("Testing 2", v, len(v))
        for idx in range(len(v)):
#             print(idx, v[idx])
#             print("Testing 1", )
            if (len(v[idx]) > 2):  # rule with more than 2 non terminal
                arr = old_prod[k].pop(idx)
                old_prod[k][idx] = []
                prev = arr[0]
                for j in range(1, len(arr)-1):
                    curr = arr[j]
                    letter = "NR" + str(count)
                    index = match(dic, label, prev, curr)
                    if index == -1:
                        old_prod[letter] = [[prev, curr]]
                        count += 1
                        dic.append([prev, curr])
                        label.append(letter)
                        prev = letter
                    else:
                        prev = label[index]
                if (k == "VP_VBN"):
                    print(["NR" + str(count-1), arr[len(arr)-1]])
                old_prod[k][idx] = ["NR" + str(count-1), arr[len(arr)-1]]
    return old_prod
    
def RemoveUnitRules(old_prod):
    ustate = list(old_prod.keys())
    flag = True
    while flag:
        flag = False
        temp_productions = copy.deepcopy(old_prod)
        for k in temp_productions:
            v = temp_productions[k]
            for idx in range(len(v)):
                if (len(v[idx]) == 1 and v[idx][0] in ustate and v[idx][0] != k):
                    flag = True
                    k2 = copy.deepcopy(v[idx][0])
                    v2 = old_prod[k2]
                    old_prod[k].remove(v[idx])
                    for j in range(len(v2)):
                        old_prod[k].append(v2[j])
    return old_prod

def RemoveDuplicate(old_prod):
    temp_prod = copy.deepcopy(old_prod)
    for k in temp_prod:
        v = temp_prod[k]
        new_v = []
        for e in v:
            if e not in new_v:
                new_v.append(e)
            else:
                old_prod[k].remove(e)
    return old_prod

def save_CNF(old_prod):
#     print(old_prod)
    with open("CNF.txt", 'w') as fp:
        for k in old_prod:
            v = old_prod[k]
            res = str(k) + " -> "
            for idx in range(len(v)):
#                 print(v[idx])
                res += str(v[idx])
                res += " | "
            res += '\n'
            fp.write(res)
        
    
"""
Assumptions and Observations
1. Eliminate start symbol from RHS. = This case is not present in given grammar, no such grammar would be given which has this case.
2. Eliminate null = This case is not present in given grammar, no such grammar would be given which has this case.
3. Eliminate terminals from RHS if they exist with other terminals or non-terminals = This case is not present in given grammar, no such grammar would be given which has this case.
4. 
"""
def CFG2CNF(old_prod):
    old_prod = pre_process_grammar(old_prod)
    old_prod = RemoveUnitRules(old_prod)
    old_prod = RemoveLargeRules(old_prod)
    old_prod = RemoveDuplicate(old_prod)
    save_CNF(old_prod)
    
    

In [24]:
# old_prod = pre_process_grammar(old_productions)
CFG2CNF(old_productions)

IndexError: list assignment index out of range

In [70]:
print(old_productions[len(old_productions)-1].lhs())

zero
